<a href="https://colab.research.google.com/github/WoradeeKongthong/raining_tomorrow_classification/blob/master/04_Raining_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Based on Feature Engineering in part 02  
Outliers : I'll cap the outliers in X_train. And cap the outliers in X_test using the boundaries of X_train.  
Missing values : I'll impute the missing values in categorical features with 'most frequent' value,  
and impute the missing values in numerical features with median.

In [0]:
# libraries
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# **Data Set**

In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/WoradeeKongthong/raining_tomorrow_classification/master/weatherAUS.csv')

In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142193 entries, 0 to 142192
Data columns (total 24 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           142193 non-null  object 
 1   Location       142193 non-null  object 
 2   MinTemp        141556 non-null  float64
 3   MaxTemp        141871 non-null  float64
 4   Rainfall       140787 non-null  float64
 5   Evaporation    81350 non-null   float64
 6   Sunshine       74377 non-null   float64
 7   WindGustDir    132863 non-null  object 
 8   WindGustSpeed  132923 non-null  float64
 9   WindDir9am     132180 non-null  object 
 10  WindDir3pm     138415 non-null  object 
 11  WindSpeed9am   140845 non-null  float64
 12  WindSpeed3pm   139563 non-null  float64
 13  Humidity9am    140419 non-null  float64
 14  Humidity3pm    138583 non-null  float64
 15  Pressure9am    128179 non-null  float64
 16  Pressure3pm    128212 non-null  float64
 17  Cloud9am       88536 non-null

In [0]:
# drop RISK_MM column (Recommendation from data description in Kaggle)
df.drop(['RISK_MM'], axis = 1, inplace = True)

# Extract Year, Month, Day from Date column
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day

# drop Date column
df.drop(['Date'], axis = 1, inplace = True)

# select year 2016-2017 to train the model
df = df[df['Year'] > 2015]

In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25974 entries, 2474 to 142192
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Location       25974 non-null  object 
 1   MinTemp        25814 non-null  float64
 2   MaxTemp        25881 non-null  float64
 3   Rainfall       25710 non-null  float64
 4   Evaporation    10704 non-null  float64
 5   Sunshine       8019 non-null   float64
 6   WindGustDir    24514 non-null  object 
 7   WindGustSpeed  24514 non-null  float64
 8   WindDir9am     24398 non-null  object 
 9   WindDir3pm     24746 non-null  object 
 10  WindSpeed9am   25841 non-null  float64
 11  WindSpeed3pm   24854 non-null  float64
 12  Humidity9am    25686 non-null  float64
 13  Humidity3pm    24218 non-null  float64
 14  Pressure9am    23294 non-null  float64
 15  Pressure3pm    23289 non-null  float64
 16  Cloud9am       14674 non-null  float64
 17  Cloud3pm       12665 non-null  float64
 18  Te

In [0]:
X = df.drop(['RainTomorrow'], axis=1)
y = df['RainTomorrow']

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [0]:
# create temp_df to combine X_train and y_train and cap the outliers
temp_df = X_train
temp_df['RainTomorrow'] = y_train

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
temp_df

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,Year,Month,Day,RainTomorrow
35832,SydneyAirport,11.7,17.4,0.0,2.2,1.9,S,57.0,SSW,S,26.0,31.0,77.0,82.0,1030.2,1028.4,6.0,7.0,15.6,16.8,No,2017,6,19,Yes
96632,Adelaide,7.9,15.8,0.0,NaN,NaN,WNW,30.0,N,WNW,4.0,11.0,96.0,56.0,1025.7,1024.9,NaN,NaN,10.2,15.3,No,2016,8,13,No
114640,PearceRAAF,12.6,27.7,0.0,NaN,10.0,WSW,31.0,E,SSW,6.0,17.0,43.0,29.0,1026.9,1022.8,NaN,NaN,20.7,26.7,No,2017,4,27,No
108793,Albany,14.2,24.2,0.0,NaN,NaN,NaN,NaN,NNW,NaN,9.0,NaN,72.0,NaN,1005.6,1003.4,7.0,NaN,17.4,NaN,No,2016,12,3,No
41082,Williamtown,6.0,22.0,0.0,3.6,NaN,NW,28.0,NW,SE,15.0,17.0,57.0,40.0,1025.0,1021.0,NaN,NaN,14.1,21.4,No,2016,8,18,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81177,Dartmoor,15.9,25.0,0.0,4.0,10.3,SE,43.0,ESE,SSE,22.0,20.0,57.0,38.0,1017.8,1017.3,NaN,NaN,19.7,24.7,No,2016,1,25,No
62351,Sale,2.5,13.7,0.0,NaN,NaN,NaN,NaN,NNW,ESE,7.0,15.0,95.0,72.0,1029.2,1026.7,8.0,NaN,8.3,13.5,No,2016,6,2,No
84326,Brisbane,21.6,30.0,0.0,8.0,13.0,NE,28.0,E,ENE,7.0,13.0,52.0,53.0,1010.4,1009.6,3.0,2.0,27.7,29.1,No,2016,1,7,No
141922,Uluru,13.2,25.7,0.0,NaN,NaN,W,28.0,ESE,E,17.0,9.0,42.0,33.0,1013.7,1011.5,8.0,8.0,18.5,25.1,No,2016,9,26,No


In [0]:
# cap the outliers in Training set

Q1 = temp_df.quantile(0.25)
Q3 = temp_df.quantile(0.75)
IQR = Q3 - Q1

lower_cap = Q1 - 1.5*IQR
upper_cap = Q3 + 1.5*IQR

features = lower_cap.index.values

for feature in features :
  temp_df[feature] = np.where(temp_df[feature]<lower_cap[feature],lower_cap[feature], temp_df[feature])
  temp_df[feature] = np.where(temp_df[feature]>upper_cap[feature],upper_cap[feature], temp_df[feature])

In [0]:
# get y_train and X_train from temp_df
X_train = temp_df.drop(['RainTomorrow'], axis=1)
y_train = temp_df['RainTomorrow']

In [0]:
# cap outliers on the test set
for feature in features :
  X_test[feature] = np.where(X_test[feature]<lower_cap[feature],lower_cap[feature], X_test[feature])
  X_test[feature] = np.where(X_test[feature]>upper_cap[feature],upper_cap[feature], X_test[feature])

# **Create Model Pipeline**

**Create Preprocessor : ColumnTransformer of numerical and categorical features**

In [0]:
numerical_features = [x for x in X.columns if df[x].dtype != 'object']

numeric_transformer = Pipeline(steps=[
          ('imputer', SimpleImputer(strategy='median')),
          ('scaler', MinMaxScaler())
])

categorical_features = [x for x in X.columns if df[x].dtype == 'object']

categorical_transformer = Pipeline(steps=[
          ('imputer', SimpleImputer(strategy='most_frequent')),
          ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
          ('num', numeric_transformer, numerical_features),
          ('cat', categorical_transformer, categorical_features)
    ]
)

**Create model**

In [0]:
from sklearn.svm import SVC
model = SVC(kernel = 'linear')

**Create Pipeline**

In [0]:
clf = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('model', model)
])

# **Cross Validation on Training set**

In [0]:
accuracy = cross_val_score(clf,X_train,y_train,cv=10)
print('accuracy : ', accuracy)
print('mean : ', accuracy.mean())
print('std : ', accuracy.std())

accuracy :  [0.83926853 0.85418672 0.83493744 0.84167469 0.8373436  0.83974976
 0.83156882 0.84408085 0.84648701 0.85074627]
mean :  0.8420043670004166
std :  0.006651902314887428


Note : 

# **Training and Test sets**

In [0]:
# Make sure your X_test is capped before running this cell

clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

print('\n\nTraining and Test Sets result')
print('\naccuracy score : ', accuracy_score(y_test,y_pred))
print('\nconfusion matrix : \n', confusion_matrix(y_test, y_pred))
print('\nclassification report : \n', classification_report(y_test,y_pred))

print('Training set score : ',clf.score(X_train,y_train))
print('Test set score : ',clf.score(X_test,y_test))



Training and Test Sets result

accuracy score :  0.8354186717998076

confusion matrix : 
 [[3778  193]
 [ 662  562]]

classification report : 
               precision    recall  f1-score   support

          No       0.85      0.95      0.90      3971
         Yes       0.74      0.46      0.57      1224

    accuracy                           0.84      5195
   macro avg       0.80      0.71      0.73      5195
weighted avg       0.83      0.84      0.82      5195

Training set score :  0.8448433514606093
Test set score :  0.8354186717998076


Note : 